## PDF Yüklemesinin Gerçekleştirilmesi

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("attentionisallyouneed.pdf")
data = loader.load()

len(data)

15

## Veriyi parçalara ayırma(Chunking İşlemi)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Chunk_Size 1000 demek her parçanın maksimum 1000 karakter uzunluğunda olacağını belirtir.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total",len(docs))

Total 52


In [4]:
docs[7]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attentionisallyouneed.pdf', 'total_pages': 15, 'page': 1, 'page_label': '2'}, page_content='in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes\nit more difficult to learn dependencies between distant positions [ 12]. In the Transformer this is\nreduced to a constant number of operations, albeit at the cost of reduced effective resolution due\nto averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as\ndescribed in section 3.2.\nSelf-attention, sometimes called intra-attention is an attention mechanism relating different positions\nof a singl

## OpenAI Generative AI Embeddings'i kullanarak Embedding Oluşturma İşlemi

In [9]:
from langchain_chroma import Chroma
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings



In [13]:
load_dotenv()

True

In [15]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

## ChromaDB Üzerine Kayıt İşlemi

In [16]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [18]:
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 10}
)

In [19]:
retieved_docs = retriever.invoke("What is encoder?")

In [20]:
len(retieved_docs)

10

In [21]:
print(retieved_docs[5].page_content)

itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding
layers, produce outputs of dimension dmodel = 512.
Decoder: The decoder is also composed of a stack of N = 6identical layers. In addition to the two
sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head
attention over the output of the encoder stack. Similar to the encoder, we employ residual connections
around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions less than i.
3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,


## OpenAI API Yapısını Kullanarak LLM Tetikleme İşlemleri

- Düşük Değerler (0.1-0.4) : Daha kesin ve daha tutarlı cevaplar verilir. Model daha tamin edilebilir hale gelir. 
- Orta Değerler (0.5-0.7) : Hem mantıklı hem de yaratıcı cevaplar verilir. 
- Yüksek Değerler (0.7-1.0): Daha rastgele ve yaratıcı, ancak bazen tutarsız yanıtlar verebilir

In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
    max_tokens=500
)

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


In [24]:
system_prompt = (
    "You are assistant for question-answering tasks"
    "Use the following pieces of retrieved context to answer"
    "If you dont't know the answer, say that you don't know"
    "Use three sentences maximum and keep the answer concise"
    "\n\n"
    "{context}"
)

In [25]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

## Soru-Cevap Zinciri Oluşturma(LLM+PROMPT)

In [26]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)

## RAG Zinciri Oluşturma(RAG+LLM Entegrasyonun Gerçekleşmesi)

In [27]:
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

## Kullanıcı Sorgusunu Çalıştırarak Cevap Üretme İşlemi

In [28]:
response = rag_chain.invoke({"input":"What is encoder?"})

In [29]:
print(response)

{'input': 'What is encoder?', 'context': [Document(id='9047b679-f654-4029-a976-8886fdb219e0', metadata={'author': '', 'creationdate': '2024-04-10T21:11:43+00:00', 'creator': 'LaTeX with hyperref', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'page': 1, 'page_label': '2', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': 'attentionisallyouneed.pdf', 'subject': '', 'title': '', 'total_pages': 15, 'trapped': '/False'}, page_content='Here, the encoder maps an input sequence of symbol representations (x1, ..., xn) to a sequence\nof continuous representations z = (z1, ..., zn). Given z, the decoder then generates an output\nsequence (y1, ..., ym) of symbols one element at a time. At each step the model is auto-regressive\n[10], consuming the previously generated symbols as additional input when generating the next.\n2'), Document(id='b4b1f05f-b009-4a0e-8b6a-8e31d0c02d87', metadata={'

In [30]:
print(response["answer"])

In the context of the Transformer model, the encoder is a component that maps an input sequence of symbol representations to a sequence of continuous representations. It consists of a stack of six identical layers, each with a multi-head self-attention mechanism and a position-wise fully connected feed-forward network. The encoder uses residual connections and layer normalization to enhance the learning process.
